In [ ]:
import pandas as pd
import numpy as np
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout
from sklearn.model_selection import StratifiedKFold, train_test_split
from bayes_opt import BayesianOptimization as bo
from tensorflow.keras import backend
import math, datetime
import itertools
%matplotlib inline

In [ ]:
n_cols = range(1, 65)
data = pd.read_csv('C:\\Users\\Aldo\\Documents\\fax\\Semestar 6\\Praksa\\final_dataset\\dataset.csv', usecols= n_cols)

In [ ]:
fraction = 0.2
dataset = data.sample(n=int(data.shape[0]*fraction), random_state=1)

In [ ]:
data_train = dataset.drop(['Y_CAT_STORNO_TOCH', 'DELTA_STORNO_FROMREZ',
                  'DELTA_STORNO_TOCH', 'STORNO_DA',  'SIF_HOTELA', 'OBRISANO_DA'], axis=1) 

In [ ]:
Ytrain = data_train.Y_CAT_STORNO_FROMREZ
Xtrain = data_train.drop('Y_CAT_STORNO_FROMREZ', axis=1)

In [ ]:
X_train, X_test, Y_train, Y_test = train_test_split(Xtrain, Ytrain, test_size=0.2, random_state=1)
Y_train = keras.utils.to_categorical(Y_train)
Y_test = keras.utils.to_categorical(Y_test)

In [ ]:
def model_function(dropout_rate=0.5):
    model = Sequential()
    model.add(Dense(512, input_shape=(57,), activation='relu'))
    model.add(Dropout(dropout_rate))
    model.add(Dense(256, activation='relu'))
    model.add(Dropout(dropout_rate))
    model.add(Dense(128, activation='relu'))
    model.add(Dense(128, activation='relu'))
    model.add(Dense(128, activation='relu'))
    model.add(Dropout(dropout_rate))
    model.add(Dense(32, activation='relu'))
    model.add(Dense(7, activation='softmax'))
    
    return model

In [ ]:
def fit_function(dropout_rate, learning_rate, epochs_rate, batch_rate):
    model = model_function(dropout_rate)
    opt = keras.optimizers.Adam(lr=learning_rate)
    model.compile(optimizer=opt, loss='categorical_crossentropy', metrics=['accuracy'])
    model.fit(X_train, Y_train, epochs=int(epochs_rate), batch_size=int(batch_rate))
    scores = model.evaluate(X_test, Y_test)
    
    return scores[1]

In [ ]:
param_bounds = {
                'dropout_rate': (0.1, 0.3), 
                'learning_rate': (1e-9, 1e-4), 
                'epochs_rate':(5, 30), 
                'batch_rate':(80, 120)
               }
with tf.Session(config=tf.ConfigProto(intra_op_parallelism_threads=16)) as sess:
    backend.set_session(sess)
    optimizer = bo(f=fit_function, pbounds=param_bounds, verbose=2, random_state=1)
    optimizer.maximize(init_points=15, n_iter=15)

In [ ]:
optimizer.max